In [2]:
import csv
import sys
import io
import re # 정규표현식 패키지
from kiwipiepy import Kiwi, Option
import tomotopy as tp
import pandas as pd

origin_path = 'C:/Users/geun/KoreanNewsChronicle/Data/201911.csv'
result_path = "C:/Users/geun/KoreanNewsChronicle/PreProcess/Data/201911_topic.csv"

origin=pd.read_csv(origin_path,encoding='utf-8')

text=origin['text']
print(text)

with open('C:/Users/geun/KoreanNewsChronicle/PreProcess/datas.txt','w',encoding='utf-8') as file:
    for item in text:
        file.write("%s\n" % item)

class Reader:
    def __init__(self, filePath):
        self.file = open(filePath,'r',encoding='UTF-8')
    
    def read(self, id):
        if id == 0: self.file.seek(0)
        return self.file.readline()

reader = Reader('datas.txt')

file = open("한국어불용어100.txt", 'r', encoding="utf-8")
stopword=[]

while True:
    line = file.readline()
    if not line: break
    wordlist = line.split('\t')
    if (wordlist[1].startswith('N')):
        stopword.append(wordlist[0])

print(stopword)
filename="datas.txt"  #파일 경로
file = open(filename, 'r', encoding="UTF-8")
kiwi=Kiwi()
kiwi.extract_add_words(reader.read)
kiwi.prepare()
stopwords = set(stopword)


def tokenize(sent): # 파일의 라인을 분석할 tokenize 함수
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용한다, 분석할때 나오는 결과에서 단어만 추출
    return [word
            for word, tag, _, _ in res
            if tag.startswith('N') and word not in stopwords] #불용어사전 적용


#LDA 모델을 적용해서 토픽 추출, k는 topic 개수
#alpha는 문헌-토픽 디리클레 분포의 하이퍼 파라미터
#eta는 토픽-단어 디리클레 분포의 하이퍼 파라미터 두개다 상수인듯하다
#min_cf는 단어의 최소 장서 빈도수, 전체 문헌내 출현빈도
#min_df는 단어의 최서 문헌 빈도수, 출현한 문헌 숫자수 의미
#tw는 용어 가중치 기법으로, ONE, IDF, PMI를 사용가능, ONE 보다는 PMI나 IDF 둘중 하나 사용 

file = open(result_path, "w", encoding='utf-8', newline = '')
writer = csv.writer(file)
writer.writerow(['index', 'topics'])
for i, line in enumerate(open(filename, encoding='utf-8')): #해당 경로의 파일을 받아와 한 라인씩 model에 추가
    model = tp.LDAModel(k=1, alpha=0.1, eta=0.001, min_cf=3,min_df=1, tw=tp.TermWeight.PMI)
    model.add_doc(tokenize(line)) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성
    model.train(0) #학습 초기화
    s = ""
    for j in range(0,100):
        model.train(20) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
    for k in range(model.k): #k는 토픽의 개수
        res = model.get_topic_words(k, top_n=10) #하위 토픽 i에 해당하는 top_n개의 단어 반환
        s = ', '.join(w for w, p in res)
    writer.writerow([i, s])
    # print(i, s)
file.close()
print("End")

topic = pd.read_csv(result_path, encoding='utf-8')

# print(csv)
# print(topic['topics'])

merge = pd.concat([origin, topic['topics']], axis=1)
print(merge)
merge.to_csv('C:/Users/geun/KoreanNewsChronicle/Topic_Abstract/Data/201911.csv', index=False, header=True, encoding="utf-8-sig")
print('merge topics')

0        시민의 생명을 위협하는 흉기난동범을 맨손으로 제압한 용감한 20대 청년 3명이 LG...
1        허창수 GS 회장이 대만의 혁신 기업인 ‘고고로’를 방문해 전기스쿠터를 직접 타보고...
2        국내 1위 편의점업체 CU를 보유한 BGF그룹이 2세 경영을 본격화한다. 홍석조 B...
3        네이버는 1일 금융 전문 자회사 ‘네이버파이낸셜’ 설립과 관련해 통장과 주식·보험,...
4        대구창조경제혁신센터. 대구창조경제혁신센터 제공대구시와 삼성전자는 지역 혁신창업기업 ...
                               ...                        
26941    이르면 내년 초부터 경기도내 아프리카 돼지열병(ASF)으로 피해를 입은 농가는 최대...
26942    로또복권 운영사 동행복권은 제887회 로또복권 추첨에서 '8, 14, 17, 27,...
26943    수도 테헤란, 이스파한 등 이란 주요 도시에서 심각한 대기오염으로 30일(현지시간)...
26944    지난 27일 黃 쓰러진 이후 동조 단식 사흘째 사무총장 “둘다 단식 중단 완강히 거...
26945    로또 운영사 동행복권은 제887회 로또복권 추첨에서 1등 당첨번호가 ‘8, 14, ...
Name: text, Length: 26946, dtype: object
['것', '수', '이', '나', '사람', '등', '우리', '때', '년', '말', '일', '때문', '그것', '일', '문제', '사회', '중', '씨', '지금', '속', '하나', '집', '월', '데', '자신', '내', '경우', '명', '생각', '시간', '그녀', '앞', '번', '여자', '개', '전', '사실', '점', '정도', '원', '소리']
End
             date press  category                                  headline  \
0   